In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

<h1 style='color:blue;' align='center'>KFold Cross Validation Python Tutorial</h2>

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

**load data and split**

In [3]:
digits = load_digits()

In [4]:
type(digits)

sklearn.utils.Bunch

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data,digits.target,test_size=0.3)

In [6]:
X_train.shape
X_test.shape

(1257, 64)

(540, 64)

**Logistic Regression**

In [7]:
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

LogisticRegression(multi_class='ovr', solver='liblinear')

0.9703703703703703

**SVM**

In [8]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

SVC(gamma='auto')

0.30185185185185187

**Random Forest**

In [9]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

RandomForestClassifier(n_estimators=40)

0.9722222222222222

<h2 style='color:purple'>KFold cross validation</h2>

**Basic example**

In [10]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [11]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


**Use KFold for our digits example**   
`StratifiedKFold` divides all classes in uniform way

In [12]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [13]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [14]:
scores_logistic

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]

In [15]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [16]:
scores_rf

[0.9248747913188647, 0.9499165275459098, 0.9248747913188647]

<h2 style='color:purple'>cross_val_score function</h2>

`cross_val_score` uses `StratifiedKFold` by default

In [17]:
from sklearn.model_selection import cross_val_score

**Logistic regression model performance using cross_val_score**

In [18]:
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), digits.data, digits.target,cv=3)

array([0.89482471, 0.95325543, 0.90984975])

**svm model performance using cross_val_score**

In [19]:
cross_val_score(SVC(gamma='auto'), digits.data, digits.target,cv=3)

array([0.38063439, 0.41068447, 0.51252087])

**random forest performance using cross_val_score**

In [20]:
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target,cv=3)

array([0.92821369, 0.93656093, 0.93322204])

<h2 style='color:purple'>Parameter tunning using k fold cross validation</h2>

Here we used `cross_val_score` to
fine tune our random forest classifier and figured that having around 40 trees in random forest gives best result. 

In [21]:
scores1 = cross_val_score(RandomForestClassifier(n_estimators=5),digits.data, digits.target, cv=10)
np.average(scores1)

0.8842675356921166

In [22]:
scores2 = cross_val_score(RandomForestClassifier(n_estimators=20),digits.data, digits.target, cv=10)
np.average(scores2)

0.9393358162631905

In [23]:
scores3 = cross_val_score(RandomForestClassifier(n_estimators=30),digits.data, digits.target, cv=10)
np.average(scores3)

0.9443699565487274

In [24]:
scores4 = cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target, cv=10)
np.average(scores4)

0.9365735567970204

<h2 style='color:purple'>Exercise</h2>

Use iris flower dataset from sklearn library and use cross_val_score against following
models to measure the performance of each. In the end figure out the model with best performance,
1. Logistic Regression
2. SVM
3. Decision Tree
4. Random Forest